# Import Dataset

In [1]:
import gc
gc.collect()
import subprocess as sp
import os
def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

get_gpu_memory()

[11170]

In [2]:
import json
from nltk.corpus import stopwords
import re
import json
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
with open('test_dataset.json', 'r') as fp:
    test_dataset = json.load(fp)
with open('train_dataset.json', 'r') as fp:
    train_dataset = json.load(fp)
f = open('/data/data_codebook.json')
data_codebook = json.load(f)
super_set={}
for s in data_codebook:
    if s[2]!="domain_name":
        if s[2] not in super_set:
            super_set[s[2]]=[]
        if s[5] not in super_set[s[2]]:
            super_set[s[2]].append(s[5])

In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix
sw = stopwords.words('english')

def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text) # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")

    text = re.sub(r"http\S+", "",text) #Removing URLs 
    #text = re.sub(r"http", "",text)
    
    html=re.compile(r'<.*?>') 
    
    text = html.sub(r'',text) #Removing html tags
    
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') #Removing punctuations
        
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    text = " ".join(text) #removing stopwords
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) #Removing emojis
    
    return text

In [4]:
def calculate_dataset_counts(dataset):
    count_dataset = {
        "general": {},
        "detailed": {}
    }
    
    for s in dataset:
        detailed_label = s['detailed_label']
        general_label = s["general_label"]
        
        if detailed_label not in count_dataset["detailed"]:
            count_dataset["detailed"][detailed_label] = 0
        count_dataset["detailed"][detailed_label] += 1

        if general_label not in count_dataset["general"]:
            count_dataset["general"][general_label] = 0
        count_dataset["general"][general_label] += 1
    
    return count_dataset

# RoBerta model

In [5]:
test_dataframe=[]
for k,v in test_dataset.items():
    for key, value in super_set.items():
        if k in value:
            super_label = key
    for s in v:
            per_line_dict = {}
            per_line_dict["sentence"] = s
            per_line_dict["detailed_label"] = k
            per_line_dict["general_label"] = super_label
            test_dataframe.append(per_line_dict)

test_dataframe = pd.DataFrame(data=(test_dataframe))
test_dataframe=shuffle(test_dataframe).dropna()
test_dataframe

,sentence,detailed_label,general_label
78983,Non-profit organizations with public particip...,Democracy,Freedom and Democracy
44918,These measures do not contribute to solving r...,Education Expansion,Welfare and Quality of Life
77212,We make it mandatory for the upper grades of ...,National Way of Life: Positive,Fabric of Society
3264,ensure that sheltered housing does not have t...,Welfare State Expansion,Welfare and Quality of Life
50432,But sand extraction in the Øresund contribute...,Environmental Protection,Welfare and Quality of Life
...,...,...,...
26960,II. Active measures to improve the well-being...,Equality: Positive,Welfare and Quality of Life
31090,PSD understands that one of the central axes ...,Culture: Positive,Welfare and Quality of Life
79145,"On that basis, the orientation towards Croati...",Democracy,Freedom and Democracy
29653,sp.a wants our society to become accessible i...,Equality: Positive,Welfare and Quality of Life


In [6]:
test_sentences_cleaned=[]
for s in list(test_dataframe["sentence"]):
    cleaned=clean_text(s)
    test_sentences_cleaned.append(cleaned)

In [10]:
import torch
import pandas as pd
from transformers import AutoTokenizer, XLMRobertaForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
# tokenize the text feature 
test_tokenized_feature = tokenizer.batch_encode_plus(
                            # Sentences to encode
                            test_sentences_cleaned, 
                            # Add '[CLS]' and '[SEP]'
                            add_special_tokens = True      
                   )
# collect tokenized sentence length 
token_sentence_length = [len(x) for x in test_tokenized_feature['input_ids']]
print('max: ', max(token_sentence_length))
print('min: ', min(token_sentence_length))
MAX_LEN=max(token_sentence_length)

max:  151
min:  2


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [27]:
# Determine the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the tokenizers for the general and detailed label identifiers
tokenizer_general = AutoTokenizer.from_pretrained('xlm-roberta-base')
tokenizer_detailed = AutoTokenizer.from_pretrained('xlm-roberta-base')

# Load the general label model weights
model_general = torch.load('Roberta_40percent_TwoStep_General', map_location=device)  # Load the model on the same device as determined above

# Load the detailed label model weights
model_detailed = torch.load('Roberta_30percent_TwoStep_Detailed', map_location=device)  # Load the model on the same device as determined above

# Put the models in evaluation mode
model_general.eval()
model_detailed.eval()

# Lists to store predicted labels
predicted_general_labels = []
predicted_detailed_labels = []

# Iterate through test data
for index, row in test_dataframe.iterrows():
    # Get the sentence and general label from the test data
    sample_sentence = row['sentence']
    cleaned_sentence=clean_text(sample_sentence)
    general_label = row['general_label']
    general_label_number = 1 if general_label == 'Welfare and Quality of Life' else 0 
    # Tokenize the sample sentence
    input_ids = tokenizer_general.encode(cleaned_sentence,  max_length = MAX_LEN, add_special_tokens=True, truncation=True, padding=True, return_tensors='pt').to(device)
    
    # Perform inference with the general label model
    with torch.no_grad():
        general_outputs = model_general(input_ids)
        general_logits = general_outputs.logits
        general_probabilities = torch.softmax(general_logits, dim=1)
        general_predicted_label = torch.argmax(general_probabilities, dim=1).item()
    
    # Append the predicted general label to the list
    predicted_general_labels.append(general_predicted_label)
    
    # Determine if the detailed model should be used based on the general label
    if general_predicted_label==1 :
        # Tokenize the sample sentence for the detailed label model
        input_ids_detailed = tokenizer_detailed.encode(cleaned_sentence,  max_length = MAX_LEN, add_special_tokens=True, truncation=True, padding=True, return_tensors='pt').to(device)
        
        # Perform inference with the detailed label model
        with torch.no_grad():
            detailed_outputs = model_detailed(input_ids_detailed)
            detailed_logits = detailed_outputs.logits
            detailed_probabilities = torch.softmax(detailed_logits, dim=1)
            detailed_predicted_label = torch.argmax(detailed_probabilities, dim=1).item()
    else:
        # If the general label doesn't match the criteria, set detailed prediction to None
        detailed_predicted_label = 0
    
    # Append the predicted detailed label to the list
    predicted_detailed_labels.append(detailed_predicted_label)

# Add the predicted labels to your test_dataframe
test_dataframe['predicted_general_label'] = predicted_general_labels
test_dataframe['predicted_detailed_label'] = predicted_detailed_labels

# Print the updated dataframe with predicted labels
print(test_dataframe.head())


                                                sentence  \
4736                                 loss of autonomy or   
52959   We will increase public awareness of environm...   
27513   Support the use of gender quotas to gain a ge...   
60994   Emphasis will be placed on preparing young pe...   
35823   —and the laws that do exist are poorly enforced.   

                        detailed_label                general_label  \
4736           Welfare State Expansion  Welfare and Quality of Life   
52959         Environmental Protection  Welfare and Quality of Life   
27513               Equality: Positive  Welfare and Quality of Life   
60994  Non-economic Demographic Groups                Social Groups   
35823          Law and Order: Positive            Fabric of Society   

       predicted_general_label  predicted_detailed_label  
4736                         0                         0  
52959                        1                         1  
27513                        1     

In [42]:
test_labels_numbers=[]
for s in list(test_dataframe["general_label"]):
    if s=='Welfare and Quality of Life':
        number=1
    else:
        number=0
    test_labels_numbers.append(number)

In [34]:
# convert numeric label to string
final_prediction_list = list(test_dataframe["predicted_detailed_label"])
# Evaluate accuracy
accuracy = accuracy_score(test_labels_numbers, final_prediction_list)

# Evaluate F1-score
f1_score = f1_score(test_labels_numbers, final_prediction_list, average='weighted')

# Evaluate balanced accuracy
balanced_accuracy = balanced_accuracy_score(test_labels_numbers, final_prediction_list)


# Print evaluation metrics for Linear SVC on the unseen test set
print("Linear SVC Accuracy on Unseen Test Set:", round(accuracy,2))
print("Linear SVC F1-Score on Unseen Test Set:", round(f1_score,2))
print("Linear SVC Balanced Accuracy on Unseen Test Set:", round(balanced_accuracy,2))
print()

# Print classification report and confusion matrix for Linear SVC on the unseen test set
print("Linear SVC Classification Report on Unseen Test Set:")
print(classification_report(test_labels_numbers, final_prediction_list))

print("Linear SVC Confusion Matrix on Unseen Test Set:")
print(confusion_matrix(test_labels_numbers, final_prediction_list))

Linear SVC Accuracy on Unseen Test Set: 0.69
Linear SVC F1-Score on Unseen Test Set: 0.61
Linear SVC Balanced Accuracy on Unseen Test Set: 0.56

Linear SVC Classification Report on Unseen Test Set:
              precision    recall  f1-score   support

           0       0.68      0.96      0.80     55471
           1       0.71      0.16      0.26     29389

    accuracy                           0.69     84860
   macro avg       0.70      0.56      0.53     84860
weighted avg       0.69      0.69      0.61     84860

Linear SVC Confusion Matrix on Unseen Test Set:
[[53529  1942]
 [24637  4752]]
